In [14]:
# import dependencies
import pandas as pd
from path import Path 
import numpy as np 
import os

In [15]:
df_columns = ['as_of_year', 'respondent_id', 'agency_name', 'agency_abbr', 'agency_code', 'loan_type_name', 'loan_type', 'property_type_name', 'property_type', 'loan_purpose_name', 'loan_purpose', 'owner_occupancy_name', 'owner_occupancy', 'loan_amount_000s', 'preapproval_name', 'preapproval', 'action_taken_name', 'action_taken', 'msamd_name', 'msamd', 'state_name', 'state_abbr', 'state_code', 'county_name', 'county_code', 'census_tract_number', 'applicant_ethnicity_name', 'applicant_ethnicity', 'co_applicant_ethnicity_name', 'co_applicant_ethnicity', 'applicant_race_name_1', 'applicant_race_1', 'applicant_race_name_2', 'applicant_race_2', 'applicant_race_name_3', 'applicant_race_3', 'applicant_race_name_4', 'applicant_race_4', 'applicant_race_name_5', 'applicant_race_5', 'co_applicant_race_name_1', 'co_applicant_race_1', 'co_applicant_race_name_2', 'co_applicant_race_2', 'co_applicant_race_name_3', 'co_applicant_race_3', 'co_applicant_race_name_4', 'co_applicant_race_4', 'co_applicant_race_name_5', 'co_applicant_race_5', 'applicant_sex_name', 'applicant_sex', 'co_applicant_sex_name', 'co_applicant_sex', 'applicant_income_000s', 'purchaser_type_name', 'purchaser_type', 'denial_reason_name_1', 'denial_reason_1', 'denial_reason_name_2', 'denial_reason_2', 'denial_reason_name_3', 'denial_reason_3', 'rate_spread', 'hoepa_status_name', 'hoepa_status', 'lien_status_name', 'lien_status', 'edit_status_name', 'edit_status', 'sequence_number', 'population', 'minority_population', 'hud_median_family_income', 'tract_to_msamd_income', 'number_of_owner_occupied_units', 'number_of_1_to_4_family_units', 'application_date_indicator']

In [16]:
# constant, float
tracking_columns = ['edit_status_name', 'edit_status', 'sequence_number']

#demographic float columns, 
demographics_columns = ['population', 'minority_population', 'hud_median_family_income', 'tract_to_msamd_income', 'number_of_owner_occupied_units', 'number_of_1_to_4_family_units']

# categorical text/numeric
applicant_sex_columns = ['applicant_sex_name', 'applicant_sex', 'co_applicant_sex_name', 'co_applicant_sex']
race_ethnicity_columns = ['applicant_ethnicity_name', 'applicant_ethnicity', 'co_applicant_ethnicity_name', 'co_applicant_ethnicity', 'applicant_race_name_1', 'applicant_race_1', 'applicant_race_name_2', 'applicant_race_2', 'applicant_race_name_3', 'applicant_race_3', 'applicant_race_name_4', 'applicant_race_4', 'applicant_race_name_5', 'applicant_race_5', 'co_applicant_race_name_1', 'co_applicant_race_1', 'co_applicant_race_name_2', 'co_applicant_race_2', 'co_applicant_race_name_3', 'co_applicant_race_3', 'co_applicant_race_name_4', 'co_applicant_race_4', 'co_applicant_race_name_5', 'co_applicant_race_5']

# loan info float
monetary_columns = ['loan_amount_000s', 'applicant_income_000s','rate_spread']

# loan info categorical
loan_info_columns = ['loan_type_name', 'loan_type', 'property_type_name', 'property_type', 'loan_purpose_name', 'loan_purpose', 'owner_occupancy_name', 'owner_occupancy', 'purchaser_type_name', 'purchaser_type','hoepa_status_name', 'hoepa_status', 'lien_status_name',
'lien_status']

# pre/approval
preapproval_columns = ['preapproval_name', 'preapproval']
approval_columns = ['action_taken_name', 'action_taken']

# denial
denial_reason_columns = ['denial_reason_name_1', 'denial_reason_1', 'denial_reason_name_2','denial_reason_2', 'denial_reason_name_3', 'denial_reason_3']

# date/time/location columns
date_time_loc_columns = ['as_of_year', 'respondent_id', 'agency_name', 'agency_abbr', 'agency_code','msamd_name', 'msamd', 'state_name', 'state_abbr', 'state_code', 'county_name', 'county_code', 'census_tract_number', 'application_date_indicator']




In [17]:
column_types = [tracking_columns, demographics_columns, applicant_sex_columns, race_ethnicity_columns, monetary_columns, loan_info_columns, preapproval_columns, approval_columns, denial_reason_columns, date_time_loc_columns]

In [18]:
# check to see the total column count
len(df_columns)

78

In [19]:
# check to make sure that all columns are accounted for
counter = 0
for item in column_types:  
    item_count = len(item)
    counter += item_count

counter

78

In [21]:
# set path and load dataframe
path = Path('./data/raw_data.csv')
df = pd.read_csv(path, low_memory=False)
df.head()